In [ ]:
!pip install pandas openpyxl
!pip install bitsandbytes==0.38.1
!pip install torch transformers==4.31.0 peft==0.4.0 sentencepiece bitsandbytes accelerate

將 Datasets 轉成 LoRA 的訓練的格式

In [ ]:
# xlsx_to_json.py
import pandas as pd
import json

def xlsx_to_json(xlsx_path, json_path):
    df = pd.read_excel(xlsx_path)

    data_list = []
    for idx, row in df.iterrows():
        article = str(row["文章"]).strip()
        question = str(row["問題"]).strip()
        option1 = str(row["選項1"]).strip()
        option2 = str(row["選項2"]).strip()
        option3 = str(row["選項3"]).strip()
        option4 = str(row["選項4"]).strip()
        correct_ans = str(row["正確答案"]).strip()  # 假設是 "1" "2" "3" "4"

        # 組成 input 區塊
        combined_input = (
            f"【文章】{article}\n"
            f"【問題】{question}\n"
            f"【選項】\n"
            f"1) {option1}\n"
            f"2) {option2}\n"
            f"3) {option3}\n"
            f"4) {option4}"
        )

        # 建立一筆 dict
        one_sample = {
            "instruction": "請仔細閱讀以下文章，理解內容後，依據邏輯分析，逐步排除錯誤選項，並最終選擇最正確的答案。只需要回覆正確的數字選項 (1, 2, 3, 或 4)。",
            "input": combined_input,
            "output": correct_ans
        }
        data_list.append(one_sample)

    # 寫入 JSON
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(data_list, f, ensure_ascii=False, indent=2)

if __name__ == "__main__":
    xlsx_path = "AI.xlsx"   # 你的原始資料
    json_path = "train.json"
    xlsx_to_json(xlsx_path, json_path)
    print(f"已將 {xlsx_path} 轉換為 {json_path}")


已將 AI1000.xlsx 轉換為 test.json


測試

In [ ]:
import torch
import transformers
from transformers import LlamaTokenizer, LlamaForCausalLM

def generate_prompt(text):
    return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{text}

### Response:"""


tokenizer = LlamaTokenizer.from_pretrained('shibing624/chinese-alpaca-plus-7b-hf')
model = LlamaForCausalLM.from_pretrained('shibing624/chinese-alpaca-plus-7b-hf').half().cuda()
model.eval()

text = '为什么天空是蓝色的？'
prompt = generate_prompt(text)
input_ids = tokenizer.encode(prompt, return_tensors='pt').to('cuda')


with torch.no_grad():
    output_ids = model.generate(
        input_ids=input_ids,
        max_new_tokens=128,
        temperature=1,
        top_k=40,
        top_p=0.9,
        repetition_penalty=1.15
    ).cuda()
output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output.replace(text, '').strip())


原始微調的code

In [ ]:
# finetune_lora.py
import os
import json
import torch
from datasets import Dataset
from transformers import (
    LlamaTokenizer,
    LlamaForCausalLM,
    TrainingArguments,
    Trainer,
)
from peft import LoraConfig, get_peft_model

# ============ A) 超參數設定 ============
BATCH_SIZE = 2
EPOCHS = 1
LR = 1e-4
MAX_LENGTH = 256
OUTPUT_DIR = "lora-out"  # 微調結果(LoRA)輸出位置

BASE_MODEL = "shibing624/chinese-alpaca-plus-7b-hf"
TRAIN_FILE = "train.json"  # 你的自訂資料集檔名

# ============ B) 讀取 JSON & 格式化資料 ============
def load_data(json_path):
    """讀取 train.json，每筆應包含 instruction, input, output三欄"""
    with open(json_path, "r", encoding="utf-8") as f:
        data_list = json.load(f)
    return data_list

def format_example(example):
    """
    - instruction: str
    - input: str (可能是空)
    - output: str (模型要學習產生的回答)
    回傳 prompt, output_text
    """
    instruction = example["instruction"]
    input_text = example.get("input", "")
    output_text = example["output"]

    if input_text.strip():
        prompt = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input_text}

### Response:"""
    else:
        prompt = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

    return prompt, output_text

def tokenize_fn(example):
    """
    把 prompt + output_text 做 tokenize，同時對 prompt 部分做 label = -100 (不計算 loss)
    """
    prompt, answer = format_example(example)
    full_text = prompt + answer

    encoding = tokenizer(
        full_text,
        max_length=MAX_LENGTH,
        truncation=True,
        padding="max_length",
    )
    # 計算 prompt 的 token 長度
    prompt_len = len(tokenizer(prompt, truncation=True, max_length=MAX_LENGTH)["input_ids"])
    labels = encoding["input_ids"].copy()

    # 將 prompt 的部分 label 設成 -100
    for i in range(prompt_len):
        if i < MAX_LENGTH:
            labels[i] = -100

    encoding["labels"] = labels
    return encoding

# ============ C) 載入 tokenizer & base model & LoRA 設定 ============
print("Loading base model & tokenizer...")
tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL)

# LLaMA/Alpaca 預設沒有 pad token，可自行加
tokenizer.add_special_tokens({"pad_token": "[PAD]"})
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids("[PAD]")

model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,  # 8-bit量化，節省顯存
    device_map="auto",
)

# LoRA 配置
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # LLaMA 常見設定
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# 讓 embeddings 大小配上新增的 [PAD] token
model.resize_token_embeddings(len(tokenizer))

# 包裝成PEFT模型
model = get_peft_model(model, lora_config)

# ============ D) 準備 Dataset ============
print("Loading dataset...")
train_data = load_data(TRAIN_FILE)
train_dataset = Dataset.from_list(train_data)
train_dataset = train_dataset.map(tokenize_fn)

# (若有 validation set，可同樣做 valid_dataset)
# valid_data = load_data("valid.json")
# valid_dataset = Dataset.from_list(valid_data).map(tokenize_fn)

# ============ E) Trainer 設定 ============
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=2,      # 或者 2, 視顯存而定
    gradient_accumulation_steps=4,      # 等效於 batch_size=8
    num_train_epochs=EPOCHS,
    learning_rate=LR,
    fp16=True,
    logging_steps=5,
    save_steps=200,
    save_total_limit=5,
    disable_tqdm=False,                 # 顯示tqdm
    logging_strategy="steps",           # 每steps都log
    report_to=[],                       # 不用wandb
)

# ============ F) 建立 Trainer & 開始微調 ============
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    # eval_dataset=valid_dataset,
)

print("Starting training...")
trainer.train()

print("Saving LoRA adapter...")
trainer.save_model(OUTPUT_DIR)
print("LoRA fine-tune is done! The adapter is saved at:", OUTPUT_DIR)

微調

In [ ]:
# finetune_lora.py
import os
import json
import torch
import gc
from datasets import Dataset
from transformers import (
    LlamaForCausalLM,
    LlamaTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model

# --------------- A) 超參數 ---------------
BATCH_SIZE = 2
EPOCHS = 5
LR = 5e-5
MAX_LENGTH = 1024
OUTPUT_DIR = "lora-out"

# --------------- B) 讀取 JSON 資料 ---------------
def load_data(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data


def format_example(example):
    """
    - instruction: str
    - input: str (可能是空)
    - output: str (模型要學習產生的回答)
    回傳 prompt, output_text
    """
    instruction = example["instruction"]
    input_text = example.get("input", "")
    output_text = example["output"]

    if input_text.strip():
        prompt = f"""請仔細閱讀以下文章，理解內容後，學習其中的邏輯，逐步排除選項，並最終選擇最正確的答案。只需要回覆正確的數字選項 (1, 2, 3, 或 4)。
        
        ### 指令:
        {instruction}

        ### 內容:
        {input_text}

        ### 最終答案:
        """
    else:
        prompt = f"""下面是描述任務的指令。編寫適當完成請求的回應。

        ### 指令:
        {instruction}

        ### 最終答案:"""

    return prompt, output_text

def tokenize_fn(example):
    prompt, answer = format_example(example)
    full_text = prompt + answer

    tokenized_prompt = tokenizer(prompt, truncation=False, return_tensors="pt")["input_ids"]
    tokenized_answer = tokenizer(answer, truncation=False, return_tensors="pt")["input_ids"]
    dynamic_max_length = min(MAX_LENGTH, len(tokenized_prompt[0]) + len(tokenized_answer[0]) + 10)

    encoding = tokenizer(
        full_text,
        max_length=dynamic_max_length,  
        truncation=True,
        padding="max_length",           # 使用填充
    )

    prompt_len = len(tokenized_prompt[0])
    labels = encoding["input_ids"].copy()
    labels[:prompt_len] = [-100] * prompt_len
    # print(f"有效 labels 数量: {sum(1 for label in labels if label != -100)}")
    
    return {"input_ids": encoding["input_ids"], "labels": labels}


# --------------- C) 載入 base model (Alpaca) ---------------
base_model_name = "shibing624/chinese-alpaca-plus-7b-hf"
tokenizer = LlamaTokenizer.from_pretrained(base_model_name)

tokenizer.add_special_tokens({"pad_token": "[PAD]"})
# 設定 tokenizer 的 pad_token_id
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids("[PAD]")

model = LlamaForCausalLM.from_pretrained(
    base_model_name,
    load_in_8bit=True,    # 利用 8-bit 量化, 節省顯存
    device_map="auto",    # 自動分配 GPU
)

# --------------- D) 設定 LoRA ---------------
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"], 
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model.resize_token_embeddings(len(tokenizer))
model = get_peft_model(model, lora_config)

# --------------- E) 準備 Dataset ---------------
train_data_list = load_data("train.json")
train_dataset = Dataset.from_list(train_data_list)
train_dataset = train_dataset.map(tokenize_fn)

# 如果有驗證集 (valid_data.json)，可同理做 valid_dataset
# valid_data_list = load_data("valid_data.json")
# valid_dataset = Dataset.from_list(valid_data_list).map(tokenize_fn)

# --------------- F) 設定 TrainingArguments ---------------
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=1,    
    gradient_accumulation_steps=32,     
    num_train_epochs=EPOCHS,
    learning_rate=LR,
    bf16=True,
    logging_steps=20,
    save_steps=3000,
    save_total_limit=1,
    disable_tqdm=False,                 # 顯示tqdm
    logging_strategy="steps",           # 每steps都log
    report_to=[],                       # 不用wandb
)

# --------------- G) 建立 Trainer 並微調 ---------------
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,  
    pad_to_multiple_of=8,  
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,  
)

for step, _ in enumerate(trainer.get_train_dataloader()):
    if step % 100 == 0:  # 定期清理缓存
        torch.cuda.empty_cache()
        gc.collect()

trainer.train()
trainer.save_model(OUTPUT_DIR)
print("LoRA fine-tune is done! The adapter is saved at:", OUTPUT_DIR)

推理

In [1]:
import pandas as pd
import csv
import torch
import re
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig
from peft import PeftModel

# ---------------------------
# 1) 載入 微調後的模型 (base + LoRA)
# ---------------------------
base_model_name = "shibing624/chinese-alpaca-plus-7b-hf"
lora_weights = "lora-out"  

tokenizer = LlamaTokenizer.from_pretrained(base_model_name)
base_model = LlamaForCausalLM.from_pretrained(
    base_model_name,
    load_in_8bit=True,  
    device_map="auto",
)
model = PeftModel.from_pretrained(base_model, lora_weights, device_map="auto")
model.eval()

def generate_prompt(instruction, input_text=""):
    return f"""請仔細閱讀以下文章，理解內容後，逐步排除選項，注意有些會有否定語句要思考仔細意思，並最終選擇最正確的答案。只需要回覆正確的數字選項 (1, 2, 3, 或 4)。

            ### Instruction:
            {instruction}

            ### Input:
            {input_text}

            ### Response:"""

def ask(instruction, input_text="", max_new_tokens=16):
    prompt = generate_prompt(instruction, input_text)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    gen_config = GenerationConfig(
        temperature=0.8,  # 提高隨機性
        top_p=0.85,       # 降低累積概率閾值，生成更多多樣答案
        num_beams=3,      # 使用多束搜索
        max_new_tokens=max_new_tokens,
    )
    
    with torch.no_grad():
        output_ids = model.generate(**inputs, generation_config=gen_config)

    output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    raw_answer = output.split("### Response:")[-1].strip()

    numbers = re.findall(r"[1234]", raw_answer)

    if numbers:
        final_answer = numbers[0]
    else:
        final_answer = "na" 

    return final_answer

# ---------------------------
# 2) 讀取測試集，進行推理
# ---------------------------
if __name__ == "__main__":
    df = pd.read_excel("AI1000.xlsx") 

    results = []

    for idx, row in df.iterrows():
        q_id = row["題號"] 
        article = str(row["文章"]).strip()
        question = str(row["問題"]).strip()
        option1 = str(row["選項1"]).strip()
        option2 = str(row["選項2"]).strip()
        option3 = str(row["選項3"]).strip()
        option4 = str(row["選項4"]).strip()

        # 組合成 input_text
        # 你可以自行修改內容格式，只要跟模型溝通清楚即可
        input_text = (
            f"【文章】{article}\n"
            f"【問題】{question}\n"
            f"【選項】\n1) {option1}\n2) {option2}\n3) {option3}\n4) {option4}"
        )

        instruction = (
            "請仔細閱讀以下文章和問題，並比較每個選項的內容。"
            "根據文章中的資訊，仔細想一想前後語句的邏輯，逐一排除不符合邏輯的選項，最後選擇最符合條件最高度正確的正確答案（1、2、3 或 4）。"
            "請按照以下步驟進行：1. 閱讀文章，理解核心內容。2. 分析問題的含義，逐一比較選項。3. 排除不符合條件的選項。4. 清晰地給出 \"最終答案：\"。"
            "有些題目會有雙重否定或是中文的語句差異，要注意這些小細節！"
            "請注意，只允許輸出一個數字答案，並避免任何解釋或多餘的字符。"
        )

        # 取得模型回答
        answers = [ask(instruction, input_text, max_new_tokens=16) for _ in range(10)]
        final_answer = max(set(answers), key=answers.count)  # 投票選出最多的答案

        print(f"ID={q_id}, Answer={final_answer}")
        results.append([q_id, final_answer])

    # ---------------------------
    # 3) 寫入 CSV 檔
    # ---------------------------
    with open("Kaggle-sample.csv", "w", newline="", encoding="utf-8") as fw:
        writer = csv.writer(fw)
        writer.writerow(["ID", "Answer"])
        writer.writerows(results)

    print("推理完成，答案已寫入 Kaggle-sample.csv")

/home/chihhung/.conda/envs/test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/chihhung/.conda/envs/test/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda116.so
CUDA SETUP: CUDA runtime path found: /home/chihhung/.conda/envs/test/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary /home/chihhung/.conda/envs/test/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda116.so...


/home/chihhung/.conda/envs/test/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/chihhung/.conda/envs/test/lib/libcudart.so.11.0'), PosixPath('/home/chihhung/.conda/envs/test/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)
/home/chihhung/.conda/envs/test/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You a

ID=10023321, Answer=4
ID=10076124, Answer=1
ID=10292442, Answer=1
ID=10450610, Answer=3
ID=10466910, Answer=2
ID=10498495, Answer=4
ID=10598939, Answer=4
ID=10657631, Answer=1
ID=10697984, Answer=4
ID=10755199, Answer=4
ID=10893831, Answer=2
ID=10899468, Answer=3
ID=11029747, Answer=2
ID=11115650, Answer=1
ID=11171068, Answer=2
ID=11264504, Answer=1
ID=11300704, Answer=1
ID=11321456, Answer=4
ID=11348435, Answer=1
ID=11368101, Answer=4
ID=11423982, Answer=1
ID=11516995, Answer=3
ID=11556898, Answer=2
ID=11693578, Answer=3
ID=11753162, Answer=3
ID=11813456, Answer=3
ID=11838237, Answer=3
ID=11891863, Answer=3
ID=12099579, Answer=1
ID=12102484, Answer=2
ID=12150342, Answer=1
ID=12246676, Answer=4
ID=12445203, Answer=2
ID=12698762, Answer=1
ID=12884075, Answer=1
ID=12926965, Answer=4
ID=13007916, Answer=4
ID=13112037, Answer=1
ID=13154559, Answer=1
ID=13258450, Answer=2
ID=13355897, Answer=3
ID=13609500, Answer=2
ID=13610809, Answer=4
ID=13691360, Answer=1
ID=13763772, Answer=3
ID=1376621